In [2]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 KB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 KB 44.0 MB/s eta 0:00:00


In [ ]:
#!pip install huggingface_hub

In [4]:
from huggingface_hub import login

login()

# Quantization
https://huggingface.co/meta-llama/Meta-Llama-3.1-70B-Instruct

In [2]:
# !pip install datasets
# !pip install llmcompressor
!pip install tf-keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.2 MB/s eta 0:00:00a 0:00:01


In [6]:
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset
from llmcompressor.transformers import SparseAutoModelForCausalLM, oneshot
from llmcompressor.modifiers.quantization import GPTQModifier
import random

model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
model_id = "meta-llama/Meta-Llama-3.1-8B"

num_samples = 256
max_seq_len = 8192

tokenizer = AutoTokenizer.from_pretrained(model_id)

def preprocess_fn(example):
  return {"text": tokenizer.apply_chat_template(example["messages"], add_generation_prompt=False, tokenize=False)}

ds = load_dataset("neuralmagic/LLM_compression_calibration", split="train")
ds = ds.shuffle().select(range(num_samples))
ds = ds.map(preprocess_fn)

recipe = GPTQModifier(
  targets="Linear",
  scheme="W8A8",
  ignore=["lm_head"],
  dampening_frac=0.1,
)

model = SparseAutoModelForCausalLM.from_pretrained(
  model_id,
  device_map="auto",
)

oneshot(
  model=model,
  dataset=ds,
  recipe=recipe,
  max_seq_length=max_seq_len,
  num_calibration_samples=num_samples,
)

model.save_pretrained("Meta-Llama-3.1-70B-Instruct-quantized.w8a8")


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


...

Removing unneeded columns:   0%|          | 0/256 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/256 [00:00<?, ? examples/s]

Map:   0%|          | 0/256 [00:00<?, ? examples/s]

2024-08-11T20:59:00.651669+0530 | one_shot | INFO - *** One Shot ***
2024-08-11T20:59:00.799090+0530 | from_modifiers | INFO - Creating recipe from modifiers
2024-08-11T20:59:00.831869+0530 | create_instance | WARNING - Could not process input as a file path or zoo stub, attempting to process it as a string.
2024-08-11T20:59:01.822409+0530 | _check_compile_recipe | INFO - Recipe compiled and 1 modifiers created
2024-08-11T20:59:01.823832+0530 | on_initialize_structure | WARNING - GPTQ quantization is set to True without an active quantization modifier.
2024-08-11T20:59:01.824278+0530 | _build_quant_modifier | INFO - Building quantization modifier with args: {'targets': 'Linear', 'scheme': 'W8A8', 'ignore': ['lm_head']}
2024-08-11T20:59:48.951624+0530 | _check_calibration_data | INFO - Skipping QuantizationModifier calibration, it is not required for the provided quantization config.
2024-08-11T21:01:17.279932+0530 | initialize_compression | INFO - Preparing model.layers.0 for compressi

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [23]:
model.

<bound method Module.modules of LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-1): 2 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): GPTQWrapper(
            (layer): Linear(
              in_features=4096, out_features=4096, bias=False
              (input_observer): MemorylessObserver()
              (weight_observer): MovingAverageMinMaxObserver()
            )
          )
          (k_proj): GPTQWrapper(
            (layer): Linear(
              in_features=4096, out_features=1024, bias=False
              (input_observer): MemorylessObserver()
              (weight_observer): MovingAverageMinMaxObserver()
            )
          )
          (v_proj): GPTQWrapper(
            (layer): Linear(
              in_features=4096, out_features=1024, bias=False
              (input_observer): MemorylessObserver()
              (weight_observer): MovingAverageMinMaxObserver()
            

In [17]:
import torch 
torch.cuda.empty_cache()

In [18]:
oneshot(
  model=model,
  dataset=ds,
  recipe=recipe,
  max_seq_length= int(max_seq_len/4),
  num_calibration_samples= int(num_samples/4),
)

model.save_pretrained("Meta-Llama-3.1-70B-Instruct-quantized.w8a8")

2024-08-11T21:37:23.743382+0530 | main | WARNING - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: False
2024-08-11T21:37:23.744367+0530 | main | INFO - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
clear_sparse_session=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_

manager stage: Model structure initialized


2024-08-11T21:37:23.921234+0530 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 1 modifiers


manager stage: Model structure initialized


2024-08-11T21:37:23.925809+0530 | pre_initialize_structure | INFO - Compression lifecycle structure pre-initialized for 1 modifiers


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

quantized_model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.bfloat16, quantization_config=quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)
input_text = "What are we having for dinner?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = quantized_model.generate(**input_ids, max_new_tokens=10)

print(tokenizer.decode(output[0], skip_special_tokens=True))


In [22]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)




quantized_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    config=config  # Pass the modified configuration
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
input_text = "What are we having for dinner?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

output = quantized_model.generate(**input_ids, max_new_tokens=10)

print(tokenizer.decode(output[0], skip_special_tokens=True))

ValueError: `rope_scaling` must be a dictionary with two fields, `type` and `factor`, got {'factor': 8.0, 'low_freq_factor': 1.0, 'high_freq_factor': 4.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'}

# This experiment of Quantization Failed

In [ ]:
!pip install auto_gptq

In [ ]:
!pip install torch


In [ ]:
!pip install optimum


In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())


In [ ]:
!pip install torch==2.0.1+cu121 torchvision==0.15.2+cu121 torchaudio==2.0.2+cu121 --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install torch==2.1.0+cu121 torchvision==0.16.0+cu121 torchaudio==2.1.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install torchtext==0.18.1 --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install torchtext==0.18.0 --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install torch==2.4.0+cu121 torchvision==0.18.1+cu121 torchaudio==2.4.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip uninstall -y torch torchvision torchaudio


In [ ]:
!pip install torch==2.4.0+cu121 torchvision==0.18.1+cu121 torchaudio==2.4.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install torch==2.4.0+cu121 torchvision==0.18.0+cu121 torchaudio==2.4.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install torch==2.3.1+cu121 torchvision==0.18.0+cu121 torchaudio==2.3.1+cu121 --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!pip install torchtext==0.18.0 --extra-index-url https://download.pytorch.org/whl/cu121


In [ ]:
!python quantization.py

In [ ]:
!nvidia-smi